# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [11]:
import pandas as pd
import numpy as np
from scipy import stats

In [12]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [13]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [73]:
sum(data[data.race=='w'].call)

235.0

In [14]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [19]:
sample_size = len(data)
if sample_size > 30:
    print("Central Limit Theorem is applied as the sample size {0} > 30".format(sample_size))

Central Limit Theorem is applied as the sample size 4870 > 30


### Forming Hypothesis
#### Null Hypothesis $H_{0}$ : There is no significant impact of race on rate of callbacks ( proportion_diff = 0). It means that both races are treated equally.
##### proportion_diff ==> difference in proportion of (race='b') getting call and (race='w') getting call
#### Alternative Hypothesis $H_{A}$ : There is significant impact of race on rate of callbacks ( proportion_diff != 0)
###### Significant Level : alpha = 0.05

#### Here I can apply two-sample test because, the point of interest is to check whether race of person has impact on call back success which is straight forward that we are dealing with two different factors

In [21]:
data.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [22]:
len(data)

4870

##### Assuming Null hypothesis true, if there is no correlation between race and call back, there should be equal probability for getting call
###### so proportion = 0.5

In [63]:
proportion_b_null = data.race.value_counts()['b']/len(data)
proportion_w_null = data.race.value_counts()['w']/len(data)
proportion_diff_null = proportion_b_null
print("proportion_diff_null ",proportion_diff_null)

proportion_diff_null  0.5


#### Now let's check the same proportions with actual call backs

In [68]:
proportion_b_alternate = sum(data[ (data['race'] == 'b') & (data['call'] == 1) ]['call']) / len(data)
proportion_w_alternate  = sum(data[ (data['race'] == 'w') & (data['call'] == 1) ]['call']) / len(data)
proportion_diff_alternate  =  proportion_b_alternate 
print("proportion_diff",proportion_diff_alternate)

proportion_diff 0.0322381930185


#### p_Value & Decision

In [70]:
sigma_prop = np.sqrt(proportion_diff_null * (1 - proportion_diff_null))/len(data)
print("sigma_prop", sigma_prop)
Zscore = (proportion_diff_alternate - proportion_diff_null) / sigma_prop
print("Zscore : ",Zscore) 

sigma_prop 0.000102669404517
Zscore :  -4556.0


In [72]:
p_Value = (1 - stats.norm.cdf(abs(Zscore))) * 2
print("p_Value : ",p_Value)

p_Value :  0.0


##### Here the p_Value is less than 0.05 which provides strong evidence against null hypothesis and provides inference that
##### there is a significant effect of race on success rate of call backs and it can be added along with the fact that number of calls received when race == b is 157 and when race == w it is 235.

### Chi-Squared Tests

In [84]:
df = data.pivot_table('id',['call'],'race',aggfunc='count')
df.index = ['Failure','Success']
df.columns = ['Black','White']
df

,Black,White
Failure,2278,2200
Success,157,235


In [86]:
chi2,p_Value,dof,expected = stats.chi2_contingency(df)

In [89]:
print("p_Value : ",p_Value)

p_Value :  4.99757838996e-05


Here also it can be observed that p_Value is less than alpha (0.05), and we can reject Null Hypothesis.

##### Here the p_Value is less than 0.05 which provides strong evidence against null hypothesis and provides inference that
##### there is a significant effect of race on success rate of call backs and it can be added along with the fact that number of calls received when race == b is 157 and when race == w it is 235.